# Homework 2 - Geospatial Analysis 

### EDS 296 

May 11, 2025 

Jordan Sibley 

## Accessing CMIP6 data 

1. Using the CMIP6 database hosted on Amazon Web Services, choose any two models you like: use both their historical simulations and future projections from one of the four major SSPs (ssp126, ssp245, ssp370, or ssp585). **Provide a brief description of the models and scenarios you chose to include.**

2. Access information from your chosen models and scenario, for any climate variable you like; however, note that three-dimensional data is generally larger and can be slower to load, so I recommend either choosing a two-dimensional data field or reading in only the surface level if you choose 3D information. Some common choices of variables to analyze, and their CMIP names, include:
- Surface air temperature (tas)
- Precipitation (pr)
- Sea surface temperature (tos)

## Choosing a study region 

3. Choose a region that you’re interested in to analyze, anywhere in the world. This should be a region that’s fairly large - think, the size of a large country or a sizable fraction of a continent. As you did for HW1, describe in markdown text some aspects of the climate of your region: **what are the interesting features there, and how might you expect that climate change would impact the area?**

## Choosing two time periods to compare

4. Choose two separate time periods, each 30-50 years in length, and describe briefly why you chose these periods: then make some maps of the time average of your selected variable.
- Map the average over each time period separately
- Map the difference in the averages between the two time periods (note: make sure to label which time period you subtracted from which!)

For both your sets of maps, display some relevant political/geographic boundaries overlaid on the region: we saw some examples of how to do this using the Cartopy “feature” toolbox in the mapping tutorials

## Results 

5. Write 1-2 paragraphs in markdown text describing the results of your plot, and what you think they might mean for humans or ecosystems located in your study region. 

INSERT TEXT HERE 